In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
#added lines
import tensorflow
from tensorflow import keras
#end of added lines
from keras.layers import Dense, SimpleRNN
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM  #class for LSTM regression
from keras.layers import Dropout
from keras.models import model_from_json
import pickle
from keras.layers import GRU
import random

In [ ]:

#global rnn_rmse, gru_rmse, lstm_rmse, regressor

sc = MinMaxScaler(feature_range = (0, 1))

In [ ]:
dataset = pd.read_csv("traffic.csv",nrows=300)
dataset["DateTime"]= pd.to_datetime(dataset["DateTime"])
dataset = dataset.drop(["ID"], axis=1) #dropping IDs

In [ ]:
dataset.shape

(300, 3)

In [ ]:
dataset["Year"]= dataset['DateTime'].dt.year
dataset["Month"]= dataset['DateTime'].dt.month
dataset["Date_no"]= dataset['DateTime'].dt.day
dataset["Hour"]= dataset['DateTime'].dt.hour

In [ ]:
print(dataset.head())
print(np.unique(dataset['Hour']))

             DateTime  Junction  Vehicles  Year  Month  Date_no  Hour
0 2015-11-01 00:00:00         1        15  2015     11        1     0
1 2015-11-01 01:00:00         1        13  2015     11        1     1
2 2015-11-01 02:00:00         1        10  2015     11        1     2
3 2015-11-01 03:00:00         1         7  2015     11        1     3
4 2015-11-01 04:00:00         1         9  2015     11        1     4
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [ ]:
temp = dataset.values
Y = temp[:,2:3]
dataset = dataset.drop(["Vehicles"], axis=1)
dataset = dataset.values
X = dataset[:,1:dataset.shape[1]]


In [ ]:
X = sc.fit_transform(X)
Y = sc.fit_transform(Y)

In [ ]:
X_train = []
y_train = []

In [ ]:
for i in range(10, 100):
    X_train.append(X[i-10:i, 0:X.shape[1]])
    y_train.append(Y[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)


In [ ]:
X_train.shape

(90, 10, 5)

In [ ]:
y_train.shape

(90,)

In [ ]:
def TrainRNN(request):
    if request.method == 'GET':
        global rnn_rmse, sc, X_train, y_train
        output = ""
        if os.path.exists('model/rnn_model.json'):
            with open('model/rnn_model.json', "r") as json_file:
                loaded_model_json = json_file.read()
                rnn = model_from_json(loaded_model_json)
            json_file.close()
            rnn.load_weights("model/rnn_model_weights.h5")
            rnn._make_predict_function()
        else:
            rnn = Sequential()
            rnn.add(SimpleRNN(units=32, input_shape=(X_train.shape[1], X_train.shape[2]), activation="relu"))
            rnn.add(Dense(8, activation="relu"))
            rnn.add(Dense(1))
            rnn.compile(loss='mean_squared_error', optimizer='adam')
            rnn.fit(X_train, y_train, epochs = 100, batch_size = 16)
            rnn.save_weights('model/rnn_model_weights.h5')
            model_json = rnn.to_json()
            with open("model/rnn_model.json", "w") as json_file:
                json_file.write(model_json)
            json_file.close()
        #performing prediction on test data
        predict_traffic = rnn.predict(X_train)
        predict_traffic = sc.inverse_transform(predict_traffic)
        predict_traffic = predict_traffic.ravel()
        y_train1 = y_train.reshape(y_train.shape[0],1)
        labels = sc.inverse_transform(y_train1)
        labels = labels.ravel()
        print("Simple RNN Predicted Growth: "+str(predict_traffic))
        print("\nOriginal Traffic: "+str(labels))
        #calculating RNN MSE
        rnn_rmse = mean_squared_error(labels,predict_traffic)
        output+='<tr><td><font size="" color="black">Simple RNN</td>'
        output+='<td><font size="" color="black">'+str(rnn_rmse)+'</td>'
        plt.plot(labels, color = 'red', label = 'Original Traffic')
        plt.plot(predict_traffic, color = 'green', label = 'Predicted Traffic')
        plt.title('Simple RNN Traffic Prediction')
        plt.xlabel('Test Data')
        plt.ylabel('Predicted Traffic')
        plt.legend()
        plt.show()
        context= {'data':output}
        return render(request, 'ViewOutput.html', context)

In [ ]:
def TrainGRU(request):
    if request.method == 'GET':
        global rnn_rmse, gru_rmse, sc, X_train, y_train
        output = ""
        if os.path.exists('model/gru_model.json'):
            with open('model/gru_model.json', "r") as json_file:
                loaded_model_json = json_file.read()
                gru = model_from_json(loaded_model_json)
            json_file.close()
            gru.load_weights("model/gru_model_weights.h5")
            gru._make_predict_function()
        else:
            gru = Sequential()
            gru.add(GRU(units=32, input_shape=(X_train.shape[1], X_train.shape[2]), activation="relu"))
            gru.add(Dense(8, activation="relu"))
            gru.add(Dense(1))
            gru.compile(loss='mean_squared_error', optimizer='adam')
            gru.fit(X_train, y_train, epochs = 700, batch_size = 16)
            gru.save_weights('model/gru_model_weights.h5')
            model_json = gru.to_json()
            with open("model/gru_model.json", "w") as json_file:
                json_file.write(model_json)
            json_file.close()
        #performing prediction on test data
        predict_traffic = gru.predict(X_train)
        predict_traffic = sc.inverse_transform(predict_traffic)
        predict_traffic = predict_traffic.ravel()
        y_train1 = y_train.reshape(y_train.shape[0],1)
        labels = sc.inverse_transform(y_train1)
        labels = labels.ravel()
        print("GRU Predicted Growth: "+str(predict_traffic))
        print("\nOriginal Traffic: "+str(labels))
        #calculating RNN MSE
        gru_rmse = mean_squared_error(labels,predict_traffic)
        output+='<tr><td><font size="" color="black">Simple RNN</td>'
        output+='<td><font size="" color="black">'+str(rnn_rmse)+'</td>'
        output+='<tr><td><font size="" color="black">GRU</td>'
        output+='<td><font size="" color="black">'+str(gru_rmse)+'</td>'
        plt.plot(labels, color = 'red', label = 'Original Traffic')
        plt.plot(predict_traffic, color = 'green', label = 'Predicted Traffic')
        plt.title('GRU Traffic Prediction')
        plt.xlabel('Test Data')
        plt.ylabel('Predicted Traffic')
        plt.legend()
        plt.show()
        context= {'data':output}
        return render(request, 'ViewOutput.html', context)


In [ ]:
def TrainLSTM(request):
    if request.method == 'GET':
        global rnn_rmse, gru_rmse, sc, lstm_rmse, regressor, X_train, y_train
        output = ""
        if os.path.exists('model/model.json'):
            with open('model/model.json', "r") as json_file:
                loaded_model_json = json_file.read()
                regressor = model_from_json(loaded_model_json)
            json_file.close()
            regressor.load_weights("model/model_weights.h5")
            regressor._make_predict_function()
        else:
            regressor = Sequential()
            regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
            regressor.add(Dropout(0.2))
            regressor.add(LSTM(units = 50, return_sequences = True))
            regressor.add(Dropout(0.2))
            regressor.add(LSTM(units = 50, return_sequences = True))
            regressor.add(Dropout(0.2))
            regressor.add(LSTM(units = 50))
            regressor.add(Dropout(0.2))
            regressor.add(Dense(units = 1))
            regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
            regressor.fit(X_train, y_train, epochs = 1000, batch_size = 16)
            regressor.save_weights('model/model_weights.h5')
            model_json = regressor.to_json()
            with open("model/model.json", "w") as json_file:
                json_file.write(model_json)
            json_file.close()
        #performing prediction on test data
        predict_traffic = regressor.predict(X_train)
        predict_traffic = sc.inverse_transform(predict_traffic)
        predict_traffic = predict_traffic.ravel()
        y_train1 = y_train.reshape(y_train.shape[0],1)
        labels = sc.inverse_transform(y_train1)
        labels = labels.ravel()
        print("LSTM Predicted Traffic: "+str(predict_traffic))
        print("\nOriginal Traffic: "+str(labels))
        #calculating LSTM RMSE
        lstm_rmse = mean_squared_error(labels,predict_traffic)
        output+='<tr><td><font size="" color="black">Simple RNN</td>'
        output+='<td><font size="" color="black">'+str(rnn_rmse)+'</td>'
        output+='<tr><td><font size="" color="black">GRU</td>'
        output+='<td><font size="" color="black">'+str(gru_rmse)+'</td>'
        output+='<tr><td><font size="" color="black">LSTM</td>'
        output+='<td><font size="" color="black">'+str(lstm_rmse)+'</td>'
        plt.plot(labels, color = 'red', label = 'Original Traffic')
        plt.plot(predict_traffic, color = 'green', label = 'Predicted Traffic')
        plt.title('LSTM Traffic Prediction')
        plt.xlabel('Test Data')
        plt.ylabel('Predicted Traffic')
        plt.legend()
        plt.show()
        context= {'data':output}
        return render(request, 'ViewOutput.html', context)
